In [ ]:
/********************老师建表数据代码************************/
/*
源码执行注意：
1.首先先建立数据库school。
2.需要建立的表有7个：student，course，studentgrade，courseteacher，
  class，deparment，teacher。
3.如果需要修改表添加约束，均需要先建立该表。
4.有些表是有外键的，外键所引用的属性所在的表需先建立。
  比如，studentgrade中的外键stu_id引用student中的主键stu_id,因此，student需先建立，且创建主键stu_id。
5.建表顺序：（1）student，course，deparment；
           （2）Student_info；
           （3）teacher，class；
           （4）courseteacher
           （5）studentgrade。

6.执行源代码时，注意上面的顺序，选择对应代码执行，不要“全”执行。
*/

--创建数据库
CREATE DATABASE school22
 ON  PRIMARY 
( NAME = 'school22_Data', 
  FILENAME = 'D:\DB\school22_Data.MDF' , 
  SIZE = 10MB , 
  MAXSIZE = UNLIMITED, 
  FILEGROWTH = 10%)
 LOG ON 
( NAME = 'school22_Log', 
  FILENAME = 'D:\DB\school22_Log.LDF' , 
  SIZE = 10MB , 
  MAXSIZE = UNLIMITED, 
  FILEGROWTH = 10%)
GO

USE school --use作用是打开数据库，接下来的操作将在这个数据库里进行。
GO

--创建student表
CREATE TABLE dbo.Student(
	Stu_id varchar(10) NOT NULL,
	Stu_name varchar(8) NOT NULL,
	Stu_sex varchar(2) NOT NULL,
	age  AS (datepart(year,getdate())-datepart(year,Birthdate)),  --计算列
	Birthdate smalldatetime NOT NULL,
	Phone1 varchar(8) NULL,
	Phone2 varchar(8) NULL,
	Address varchar(100) NULL,
	Class_id varchar(4) NULL,
 CONSTRAINT PK_Student PRIMARY KEY CLUSTERED (Stu_id ASC)    --主键，聚集索引（clustered）
 ) 

--通过修改表的方式，添加默认值约束
ALTER TABLE dbo.Student 
ADD  CONSTRAINT DF_Student_Stu_sex  DEFAULT ('男') FOR Stu_sex

--创建表course
CREATE TABLE dbo.Course(
	Course_id varchar(6) NOT NULL,
	Course_name varchar(20) NOT NULL,
	Course_hour smallint NOT NULL,
	Introduce varchar(200) NULL,
 CONSTRAINT PK_Course PRIMARY KEY CLUSTERED  (Course_id ASC)
) 

--修改表，添加默认值约束
ALTER TABLE dbo.Course 
ADD  CONSTRAINT DF_Course_Course_hour  DEFAULT (60) FOR Course_hour

--创建表studentgrade。注意，这个表创建之前，需要先建立student和course。

CREATE TABLE dbo.StudentGrade(
	Stu_id varchar(10) NOT NULL,
	Course_id varchar(6) NOT NULL,
	Grade smallint NULL,
 CONSTRAINT PK_StudentGrade PRIMARY KEY CLUSTERED (Stu_id ASC,Course_id ASC)
)

--修改表，添加外键约束
ALTER TABLE dbo.StudentGrade  
ADD  CONSTRAINT FK_StudentGrade_Course FOREIGN KEY(Course_id) REFERENCES dbo.Course (Course_id)

--修改表，添加外键约束
ALTER TABLE dbo.StudentGrade 
ADD  CONSTRAINT FK_StudentGrade_Student FOREIGN KEY(Stu_id)
REFERENCES dbo.Student (Stu_id)

--创建表deparment
CREATE TABLE dbo.Deparment(
	Depar_id varchar(2) NOT NULL,
	Depar_name varchar(50) NULL,
	Phone varchar(8) NULL,
	teac_num int NULL,
 CONSTRAINT PK_TABLE1 PRIMARY KEY CLUSTERED (Depar_id ASC)
) 

--修改表，添加check约束
ALTER TABLE dbo.Deparment  
ADD  CONSTRAINT CK_Deparment_Phone CHECK ((phone like '8521[0-9][0-9][0-9][0-9]'))

--创建表teacher
CREATE TABLE dbo.Teacher(
	Teac_id varchar(6) NOT NULL,
	Teac_name varchar(8) NOT NULL,
	Teac_sex varchar(2) NULL,
	Birthdate smalldatetime NULL,
	TechPost varchar(10) NULL,
	Depar_id varchar(2) NULL,
 CONSTRAINT PK_Teacher PRIMARY KEY CLUSTERED (Teac_id ASC)
) 

--修改表，添加默认值约束
ALTER TABLE dbo.Teacher 
ADD  CONSTRAINT DF_Teacher_Teac_name  DEFAULT ('讲师') FOR Teac_name

--修改表，添加外键
ALTER TABLE dbo.Teacher 
ADD  CONSTRAINT FK_Teacher_Deparment FOREIGN KEY(Depar_id) REFERENCES dbo.Deparment (Depar_id)

--创建class
CREATE TABLE dbo.Class(
	Class_id varchar(4) NOT NULL,
	Class_name varchar(20) NOT NULL,
	Director varchar(6) NULL,
	Monitor varchar(10) NULL,
	Depar_id varchar(2) NULL,
	class_num int NULL,
 CONSTRAINT PK_Class PRIMARY KEY CLUSTERED (Class_id ASC)
) 

--修改表，添加外键
ALTER TABLE dbo.Class 
ADD  CONSTRAINT FK_Class_Deparment FOREIGN KEY(Depar_id) REFERENCES dbo.Deparment (Depar_id)

--创建表courseteacher
CREATE TABLE dbo.CourseTeacher(
	Course_id varchar(6) NOT NULL,
	Class_id varchar(4) NOT NULL,
	Teac_id varchar(6) NOT NULL,
 CONSTRAINT PK_CourseTeacher PRIMARY KEY CLUSTERED 
(Course_id ASC,	Class_id ASC,	Teac_id ASC )
)

--修改表，添加外键
ALTER TABLE dbo.CourseTeacher 
ADD  CONSTRAINT FK_CourseTeacher_Class FOREIGN KEY(Class_id) REFERENCES dbo.Class (Class_id)

--修改表，添加外键
ALTER TABLE dbo.CourseTeacher  
ADD  CONSTRAINT FK_CourseTeacher_Course FOREIGN KEY(Course_id) REFERENCES dbo.Course (Course_id)

--修改表，添加外键
ALTER TABLE dbo.CourseTeacher 
ADD  CONSTRAINT FK_CourseTeacher_Teacher FOREIGN KEY(Teac_id) REFERENCES dbo.Teacher (Teac_id)

--创建表student_info
CREATE TABLE dbo.Student_info(
	Stu_id varchar(50) NOT NULL,
	Award varchar(500) NULL,
	Punish varchar(500) NULL,
 CONSTRAINT PK_Student_info PRIMARY KEY CLUSTERED (Stu_id ASC)
)

select * from CourseTeacher

In [ ]:
/*********************老师参考答案**********************/
--1. 查询学生的姓名、性别、班级编号，并把结果存储在一张新表中。（提示：用Select……Into……）
SELECT stu_name,stu_sex,class_id INTO newstudent
FROM Student

--2．查询男生的资料。
SELECT *
FROM Student
WHERE Stu_sex='男'

--3．查询所有计算机系的班级信息。
--方法一、WHERE连接查询
SELECT *
FROM Class,Deparment
WHERE Class.depar_id=Deparment.depar_id  AND Depar_name='计算机系'
--方法二、JOIN连接查询
SELECT *
FROM Class JOIN Deparment ON Class.depar_id=Deparment.depar_id
WHERE  Depar_name='计算机系'
--方法三、嵌套查询
SELECT *   --父查询：根据子查询找出的系别编号，查找该系别编号的班级
FROM Class
WHERE Depar_id=(SELECT Depar_id   --子查询：找出计算机系的系别编号
                FROM Deparment
                WHERE Depar_name='计算机系')

--4．查询年龄小于30岁的女同学的学号和姓名。
--GETDATE()为获取系统时间的函数，YEAR()为获取“年”,MONTH()为获取“月”,DAY()为获取“日”。
SELECT *
FROM Student
WHERE (YEAR(GETDATE())-YEAR(Birthday))<30 AND Stu_sex='女'


--5．查询艾老师所教的课程号。(也可用嵌套查询)
SELECT Course_id,*
FROM CourseTeacher JOIN Teacher ON CourseTeacher.Teac_id=Teacher.Teac_id
WHERE Teac_name like '艾%'


--6．在学生基本信息表Student中检索学生的姓名和出生年份，输出的列名为STUDENT_NAME和BIRTH_YEAR。
SELECT Stu_name AS STUDENT_NAME,YEAR(Birthday) AS BIRTH_YEAR   --AS作用是起别名
FROM Student

--7．在StudnetGrade中，求选修课程“0511”且分数排名为前10%学生的学号和得分。
SELECT TOP 10 PERCENT  Stu_id,Grade  
FROM StudentGrade
WHERE Course_id='0511'
ORDER BY Grade DESC

--若考虑存在并列（正好分界线上的成绩相同）排名的情况，可用TOP 10 PERCENT with ties，保留相同值的记录。  或者：
SELECT Stu_id,Grade
FROM  StudentGrade
WHER Grade in (SELECT TOP 10 PERCENT Grade  
    FROM StudentGrade
    WHERE Course_id='0511'
    ORDER BY Grade DESC
    )
AND Course_id='0511'

--8．查询“张三”的学号。
SELECT Stu_id
FROM Student
WHERE Stu_name='张三'

--9．查询课程“0101”的成绩在80与90之间的同学的学号。
SELECT Stu_id
FROM StudentGrade
WHERE Course_id='0101' AND (Grade BETWEEN 80 AND 90)

In [ ]:
/********************我的答案********************/
--1.把学生姓名，性别，班级编号传入新表
SELECT Stu_name,Stu_sex,Class_id,stu_id  into Newtest FROM Student
select * from  Newtest
drop table newtest
--2.查询男生信息
SELECT * FROM student where Stu_sex ='男'

--3.查询所有选修课程号，课程号不重复
select distinct([Course_id]) from [dbo].[StudentGrade]

--4.获取年龄小于30的女生的学号和姓名
select Stu_id,Stu_name from student where year(getdate())-year([Birthdate])<=30

--5.在学生基本表中查询学生的姓名和出生年份，命名为STUDENT_NAME,BIRTH_YEAR
SELECT Stu_name STUDENT_NAME,Birthdate BIRTH_YEAR from student

--6.在StudentGrade 找出‘0511’课程号且分数排名为10%的学号和成绩
select top 10 percent Stu_id,Grade from StudentGrade where [Course_id]='C0511'
order by Grade asc

--7.查询课程号为'0109'或‘0111’的学生学号    --也可以用union
select Stu_name ,Course_id from Student ,StudentGrade where Student.Stu_id=StudentGrade.Stu_id 
and (Course_id='C0109' or Course_id='C0111')

--8.查询课程号为'0109'和‘0111’的学生学号    --也可以用交运算或嵌套查询
select  A.Stu_id , A.Course_id,B.Course_id from StudentGrade A inner join StudentGrade B 
on  A.stu_id= B.stu_id
where ( A.Course_id = 'C0109' and  B.Course_id ='C0111')

--9.查询‘0101’课程，成绩在80-90学生学号
select grade ,Stu_id from StudentGrade where Course_id ='C0111' and grade between 80 and 90

--10.查询平均成绩在80以上的学生学号和平均成绩
select stu_id ,avg(grade) avg_grade from  StudentGrade group by Stu_id having avg(grade)>=80